In [ ]:
!pip install -q sentence-transformers pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 62.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 46.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 32.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 79.7 MB/s eta 0:00:00


In [ ]:
import pandas as pd

filename = "ebay_RTX 4070_2025-04-22"
df = pd.read_csv(f"{filename}.csv")
df['text'] = df['Title'].fillna('') + ' ' + df['Item Description'].fillna('')
df.head()


,Title,Item Description,Total Price,Link,text
0,NaN,NaN,$0.00,https://ebay.com/itm/123456?itmmeta=012DEW30YG...,
1,NaN,NaN,$0.00,https://ebay.com/itm/123456?itmmeta=012DEW30YG...,
2,"Lenovo LOQ 15.6"" FHD 144Hz Gaming Notebook R7-...","Lenovo LOQ 15 15.6"" 1920 x 1080 FHD 144H...",$979.99,https://www.ebay.com/itm/305718430632?_skw=RTX...,"Lenovo LOQ 15.6"" FHD 144Hz Gaming Notebook R7-..."
3,"Acer Predator Helios 16""Laptop NVIDIA GeForce ...",Unleash unparalleled gaming power with the Ace...,$1200.00,https://www.ebay.com/itm/267072411765?_skw=RTX...,"Acer Predator Helios 16""Laptop NVIDIA GeForce ..."
4,NEW LISTINGGIGABYTE GeForce RTX 4070 WINDFORCE...,Card works great. Has one dent on the backplat...,$550.00,https://www.ebay.com/itm/335924302172?_skw=RTX...,NEW LISTINGGIGABYTE GeForce RTX 4070 WINDFORCE...


In [ ]:
positive_prompt = "This is a desktop NVIDIA GeForce RTX 4070 graphics card"


In [ ]:
from sentence_transformers import SentenceTransformer, util

model = SentenceTransformer('all-MiniLM-L6-v2')  # lightweight but good

# Encode both prompt and listings
prompt_embedding = model.encode(positive_prompt, convert_to_tensor=True)
listing_embeddings = model.encode(df['text'].tolist(), convert_to_tensor=True)

# Compute cosine similarities
cosine_scores = util.cos_sim(prompt_embedding, listing_embeddings)[0]


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

OSError: We couldn't connect to 'https://huggingface.co' to load the files, and couldn't find them in the cached files.
Checkout your internet connection or see how to run the library in offline mode at 'https://huggingface.co/docs/transformers/installation#offline-mode'.

In [ ]:
threshold = 0.55
df['similarity'] = cosine_scores.cpu().numpy()
df['is_exclusive_4070'] = df['similarity'] > threshold # adjust threshold as needed

# See results
filtered_df = df[df['is_exclusive_4070']]
filtered_df[['title', 'price', 'similarity', 'eBay link']]


In [ ]:
filtered_df.to_csv("filtered_ebay_RTX 4070_2025-04-13.csv", index=False)
from google.colab import files
files.download("filtered_4070_listings.csv")
